In [1]:
# Python 2
import pickle
with open('df.pickle', mode='rb') as h:
    df = pickle.load(h)
with open('hex.pickle', mode='rb') as h:
    label_count = len(pickle.load(h)['id_name'])

In [2]:
import cv2
import numpy as np
from os.path import join
img_dir = '../pascal12/JPEGImages'
img_mean = np.load('../ilsvrc_2012_mean.npy').astype(np.float32)  # 3*256*256
img_mean = np.swapaxes(np.swapaxes(img_mean, 0, 1), 1, 2)  # convert to XY[BGR]
def load_image(filename):
    """
    Loads image, subtract mean, resize to 227*227. No axis rotation or normalization.
    Returns:
        Result image as an np.float32 array with size 227*227*3. Values within [0, 255].
    """
    img = cv2.resize(cv2.imread(join(img_dir, filename)), (256, 256))
    return cv2.resize(img.astype(np.float32) - img_mean, (227, 227))

In [3]:
def get_split_XY(split):
    df_split = df[df['split'] == split]
    X = map(load_image, df_split['img'])
    Y = np.zeros((len(X), label_count), dtype=np.float32)
    pseudo_labels = df_split['pseudo_label'].reset_index(drop=True)
    for i in range(0, len(X)):
        Y[i, pseudo_labels[i]] = 1.0 / len(pseudo_labels[i])  # softmax + cross entropy loss
    return np.array(X, dtype=np.float32), Y

In [ ]:
import h5py
def write_split_XY(split, X, Y):
    with h5py.File('../{}.h5'.format(split), mode='w') as h:
        h.create_dataset('data', data=X)
        h.create_dataset('label', data=Y)

In [ ]:
X_train, Y_train = get_split_XY('train')
X_train = np.swapaxes(np.swapaxes(X_train, 2, 3), 1, 2)  # convert to [BGR]XY
write_split_XY('train', X_train, Y_train)
X_test, Y_test = get_split_XY('test')
write_split_XY('test', X_test, Y_test)